In [1]:
import numpy as np

### Step 1: Define the robot kinematics

#### x = f(q)


### Step 2: Compute Jacobian for each task or constraint

#### dxi = Ji(q) dq,   where i = 0, 1,2 ... .k-1

 and J0 has the higest priority and Jk-1 has the lowest proiority

### Step 3: Apply psuedo inverse Jacobian for each task/constraint since dim(xi) < dim(q) for real world application
<img src="test.png">

 Weigted psuedo inverse could be applied depending upon different joint types. But for the simplicity, we will use Moore-Penrose psuedo inverse here

### Step 4: Compute the null space N of Jacobian J
<img src = "null.png">


### Step 5: Compute successively nullspace projection matrix. Less import tasks are projected recursively onto the nullspace Ni.
<img src = "form4.png">

### Step 6: Calculate the joint velocity with hierarchical multiple tasks:

### dq = sum(Ni,dqi) where i =0,k-1

In [2]:
def compute_inverse_hierarchy(dx0_J0,*argv):

    """Compute successive nullspace projection matrics
    input: dx0_J0,tuple of (dx0, J0)
            argv, tuple of (dxi, Ji) where i = i ... k-1
    output: dq for hirarachical multiple tasks
    """

    # Determine the null space of the highest proiority task via psuedo inverve first
    dx0, J0 = dx0_J0
    m0  =  J0.shape[1]
    J0_pinv = np.linalg.pinv(J0)
    H0 = np.matmul(J0_pinv,J0)
    null = np.identity(m0) - H0
    dq_null = np.zeros((m0,1))
    
    # Iteratively commpute the null space of each Jacobian(Ji) and task points(dxi)
    for dxi,Ji in argv:
        Ji_pinv = np.linalg.pinv(Ji) 
        Hi = np.matmul(Ji_pinv,Ji)

        mi = Ji.shape[1]  # m is the number of constrains given task [i]
        null = np.matmul(null, (np.identity(mi) - Hi))

        dqi = np.matmul(Ji_pinv,dxi)
        dq_null = dq_null+ np.matmul(null,dqi)
        
    dq = np.matmul(J0_pinv,dx0) + dq_null
    
    return dq,dq_null



## Step 7: Check the algortihm  with  a simple example 

Let's consider a planar robot with 3 links. Each link has unit length 1.
Its forward kinematics is defined as 
#### x = cos(q1)+ cos(q1+q2) + cos(q1+q2+q3)
#### y = sin(q1)+ sin(q1+q2) + sin(q1+q2+q3)

The  Jacobian of the 3rd link tip  can be computed  as the following when when q1 = 0 and q2 = 90 and q3 = 0.  
#### J0 =[ [-1-1, 0], [2, 1,1]] 

The high priority task is defined to follow 
#### dx0= [0,1]T

In [3]:
J0 = np.array([[-1, -1, 0],[2,1,1]])
dx0 = np.array([[1],[0]])

J0_pinv = np.linalg.pinv(J0)
J0_pinv

array([[-5.87388775e-17,  3.33333333e-01],
       [-1.00000000e+00, -3.33333333e-01],
       [ 1.00000000e+00,  6.66666667e-01]])

The  Jacobian of the 2nd link tip at the same configureation is 
#### J0 =[ [-1-1, 0], [1,1,0] 
The low priority task is to follow 
#### dx1= [1,1]T

In [4]:
J1 = np.array([[ 1, -1,0],[1,0,0]])
dx1 = np.array([[0],[1]])

J1_pinv = np.linalg.pinv(J0)
J1_pinv

array([[-5.87388775e-17,  3.33333333e-01],
       [-1.00000000e+00, -3.33333333e-01],
       [ 1.00000000e+00,  6.66666667e-01]])

In [5]:
dx0_J0 = (dx0, J0)
dx1_J1 = (dx1,J1)

In [6]:
dq, dq_null = compute_inverse_hierarchy(dx0_J0, dx1_J1)
print("dq",dq)
print("dq_null", dq_null)

dq [[-1.03728349e-16]
 [-1.00000000e+00]
 [ 1.00000000e+00]]
dq_null [[-4.49894711e-17]
 [ 4.49894711e-17]
 [ 4.49894711e-17]]


### ^^ dq_null is all zeros. Something wrong!!!

## Sandbox: Ignore below

In [7]:
dx0, J0 = dx0_J0
m0  =  J0.shape[1]
J0_pinv = np.linalg.pinv(J0)
H0 = np.matmul(J0_pinv,J0)
null = np.identity(m0) - H0
dq_null = np.zeros((m0,1))
null   
    # Iteratively commpute the null space of each Jacobian(Jk) and task points(dxk)


array([[ 0.33333333, -0.33333333, -0.33333333],
       [-0.33333333,  0.33333333,  0.33333333],
       [-0.33333333,  0.33333333,  0.33333333]])

In [8]:
J1_pinv = np.linalg.pinv(J1)
H1 = np.matmul(J1_pinv,J1)
m1 = J1.shape[1]  # m is the number of constrains given task [i]
null = np.matmul(null, (np.identity(m1) - H1))

dq1 = np.matmul(J1_pinv,dx1)
dq_null = dq_null+ np.matmul(null,dq1)
        
dq = np.matmul(J0_pinv,dx0) + dq_null


In [9]:
print (np.identity(m1) - H1)

[[0.00000000e+00 8.70761916e-17 0.00000000e+00]
 [0.00000000e+00 2.22044605e-16 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [10]:
print(null)

[[ 0.00000000e+00 -4.49894711e-17 -3.33333333e-01]
 [ 0.00000000e+00  4.49894711e-17  3.33333333e-01]
 [ 0.00000000e+00  4.49894711e-17  3.33333333e-01]]


In [11]:
dq1

array([[1.],
       [1.],
       [0.]])

In [12]:
dq0

NameError: name 'dq0' is not defined

In [ ]:
np.matmul(null,dq1)

In [ ]:
dq0 = np.matmul(J0_pinv,dx0)
dq0

In [ ]:
H0 = np.matmul(J0_pinv, J0)
H0

In [ ]:
N0 = np.identity(3) - H0
N0

In [ ]:
dq0 = np.matmul(J0_pinv,dx0)

In [ ]:
null = np.identity(3) - H0

In [ ]:
null

In [ ]:
dq1 = np.matmul(J1_pinv,dx1)
dq1

In [ ]:
dq_null = np.matmul(N0,dq1)
dq_null